# Exercício 01 - Perceptron Adaline

## Aluno
- Moisés Botarro Ferraz Silva, 8504135

# Objetivo

Nesse Exercício, iremos implementar o modelo de Perceptron Adaline, utilizando a Regra Delta para ajuste dos pesos. O objetivo é treinar o modelo e utilizá-lo em seguida para reconhecer os símbolos Y e Y invertido.

# Dados

Foram criados 12 arquivos para treinamento: 6 da classe Y (a qual chamaremos de +1) e 6 da classe Y invertido (-1). Esses arquivos encontram-se dentro da pasta **training**. Cada arquivo consiste de um csv com os números -1 e +1 formando a letra Y ou Y invertido. O tamanho do arquivo foi fixado em 5x5.

Para a etapa de teste, também foram gerados 6 exemplos, 3 para cada classe. Estes encontram-se no diretório **teste**.


## Leitura dos dados

In [1]:
import numpy as np
import csv
import os

# readFile reads a csv file with the image and return a 1D flatten array with its content
def readFile(filename):
    with open(filename) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        img = []
        for row in readCSV:
            img.append(row)
        
    return np.array(img, dtype=int).flatten()

# getData reads all csv files inside a directory.
def getData(directory):
    data = list()
    for root, dirs, files in os.walk(directory):
        # iterate through the files in the given directory
        for f in files:
            if f.endswith(".csv"):
                print('reading file ' + directory + '/' + f)
                data.append(readFile(directory + '/' + f))
                
    return np.array(data)

## Dados de treinamento

Vamos criar uma matriz **train_data** com todos os exemplos utilizados para o treinamento, com cada exemplo ocupando uma linha. O vetor **train_classes** contém a classe de cada exemplo na correspondente linha de train_data. 

Assim, o exemplo da linha 1 de train_data, terá sua classe informada na posição 1 de train_classes e assim sucessivamente.

In [2]:
# train_classes has the class of each training sample
train_classes = []

# read the Y examples and add their class (+1) in the class list
train_y = getData("training/y")
train_classes.extend([1 for i in range(len(train_y))])

# read the inverted Y examples and add their class (-1) in the class list
train_y_inv = getData("training/y_inv")
train_classes.extend([-1 for i in range(len(train_y_inv))])

# Concatenate the train_y with train_y_inv examples
train_data = np.concatenate((train_y, train_y_inv))

reading file training/y/y4.csv
reading file training/y/y5.csv
reading file training/y/y6.csv
reading file training/y/y2.csv
reading file training/y/y3.csv
reading file training/y/y1.csv
reading file training/y_inv/y_inv1.csv
reading file training/y_inv/y_inv2.csv
reading file training/y_inv/y_inv3.csv
reading file training/y_inv/y_inv6.csv
reading file training/y_inv/y_inv4.csv
reading file training/y_inv/y_inv5.csv


In [3]:
# The train_data
train_data

array([[-1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1, -1,  1, -1, -1, -1,
        -1,  1, -1, -1, -1, -1,  1, -1, -1],
       [-1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1,
        -1,  1, -1, -1, -1, -1,  1, -1, -1],
       [ 1, -1, -1, -1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1,
        -1,  1, -1, -1, -1, -1,  1, -1, -1],
       [ 1, -1, -1, -1,  1, -1,  1, -1,  1, -1, -1, -1,  1, -1, -1, -1,
        -1,  1, -1, -1, -1, -1,  1, -1, -1],
       [-1,  1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1, -1, -1, -1,
        -1,  1, -1, -1, -1, -1,  1, -1, -1],
       [ 1, -1, -1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1,
        -1,  1, -1, -1, -1, -1,  1, -1, -1],
       [-1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1, -1,  1,
        -1, -1, -1,  1,  1, -1, -1, -1,  1],
       [-1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1,
         1, -1,  1, -1,  1, -1, -1, -1,  1],
       [-1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, 

In [4]:
# Its classes
train_classes

[1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1]

# Perceptron 

## Modelo

Iremos utilizar o modelo de Perceptron apresentado na figura à seguir.

<img src="images/modelo_perceptron.png" alt="modelo perceptron" width="600"/>

Nele, consideramos uma entrada como bias. Seu valor será sempre 1 e o bias será ajustado como os pesos das demais entradas.

Para calcular a saída do neurônio, utilizaremos a expressão

$$ y = f(\sum_{i} w_i x_i - \theta)$$

onde f é a função sinal, definida como

$$ 
    f(x)= 
\begin{cases}
    1,& \text{se } x\geq 0\\
    -1,              & \text{caso contrário}
\end{cases}
$$

In [5]:
# signal function definition
def f_signal(x):
    if x >= 0:
        return 1
    else:
        return -1

## Aprendizado

Utilizaremos a Regra Delta para treinamento do Perceptron. Ela consiste das seguintes etapas:
    1. Inicialização dos pesos sinápticos com valores randômicos pequenos ou iguais a zero
    2. Aplicação de um padrão com seu respectivo valor esperados de saída t e verificação da saída da rede y
    3. Cálculo do erro de saída E = t - j
    4. Se E == 0, volta-se ao passo 2
       Se E != 0, atualiza-se os pesos
    5. Volta-se ao passo 2

Para a atualização de cada peso $ w_i$ referente à entrada i, utiliza-se a seguinte fórmula

$$ \Delta w_i = \eta(t - j)x_i , $$


obtida através do gradiente da função erro em relação aos pesos adotados nas entradas. A constante $ \eta $ é conhecida como a taxa de aprendizado e adoremos o valor 0,5 no nosso exemplo.

Durante a fase de treinamento, os exemplos serão apresentados em ciclos. O treinamento é finalizado quando, ao final de um ciclo, todas as imagens de treinamento forem classificadas corretamente.

In [6]:
# Perceptron is a class that represents a single Perceptron. It has a method train to adjust its weigths according
# to the given training examples. It also has a predict method to predict the classes of a list of examples passed 
# to it as a parameter. 
class Perceptron:
    def __init__(self, input_size):
        # Perceptron parameters'  initialization
        self.bias = np.random.rand()
        self.weights = np.random.rand(input_size)
        
    # compute_output computes the output y for a given sample presented to the perceptron inputs
    def compute_output(self, inputs):
        return f_signal(np.dot(inputs,self.weights) - self.bias)

    
    # train adjusts the weights of the perceptron inputs and bias for a given training set
    # parameters:
    #             samples: matrix with each input example in a line
    #             classes: array with the samples classes
    #             eta: learning rate
    def train(self, samples, classes, eta = 0.5):
        changed_weight = False
        cycle = 1
        
        # The training continues until there's no weight adjust in a cycle
        while changed_weight == True or cycle == 1:
            print("Início do Ciclo %d" % cycle)
            
            changed_weight = False
            
            # Present each one of the training examples to the Perceptron
            for sample in range(len(samples)):

                t = classes[sample]              # example true class
                inputs = samples[sample]         # example content in a 1D array
                y = self.compute_output(inputs)  # example predicted class

                # Only update the weigths if the output y is different from the true class t
                if (y != t):
                    changed_weight = True
                    
                    print("\tA imagem %d (classe %d) não  foi classificada corretamente! " % (sample, t), end=" ")
                    print("Ajustando pesos!")

                    # Update input weights
                    for i in range(0, len(self.weights)):
                        deltaWi = eta*(t-y)*inputs[i]
                        self.weights[i] = self.weights[i] + deltaWi
                        
                    # Update bias weight
                    deltaBias = eta*(t-y)*1    # the bias input is always 1
                    self.bias = self.bias + deltaBias
            
            cycle = cycle + 1 
        
        
    # predict infers the class for each one of the examples given in the samples matrix
    def predict(self, samples):
        predicted_classes = list()
        
        for s in samples:
            predicted_classes.append(self.compute_output(s))
            
        return predicted_classes

# Fase de Treinamento

Para a etapa de aprendizado, serão utilizados os arquivos lidos do conjunto de treinamento. Um Perceptron será criado com o mesmo número de entradas que o conteúdo dos arquivos com as letras Y e Y invertido (5x5 = 25 entradas).

In [8]:
perceptron = Perceptron(5*5)
perceptron.train(train_data, train_classes, 0.5)

Início do Ciclo 1
	A imagem 0 (classe 1) não  foi classificada corretamente!  Ajustando pesos!
Início do Ciclo 2


Iremos criar uma função de acurácia para avaliação do modelo treinado. A acurácia do modelo será calculada como

$$ acurácia = \frac{\text{número de exemplos classificados corretamente}}{\text{número total de exemplos classificados}} $$ 

In [9]:
def get_accuracy(predicted, real):
    correct = 0
    for i in range(len(predicted)):
        if predicted[i] == real[i]:
            correct = correct+1
            
    return correct/len(predicted)

Utilizaremos o Perceptron para classificar os exemplos usados na fase de treinamento. Esperamos obter uma acurácia de 100%, uma vez que o treinamento foi finalizado apenas quando o modelo classificou corretamente todos os exemplos.

In [10]:
predicted_classes = perceptron.predict(train_data)
train_acc = get_accuracy(predicted_classes, train_classes)
print("Acurácia para exemplos de treinamento: %.2f%%" % (train_acc*100))

Acurácia para exemplos de treinamento: 100.00%


# Fase de Teste

Leitura dos exemplos de teste:

In [11]:
# test_classes has the class of each test sample
test_classes = []

# read the Y examples and add their class (+1) in the class list
test_y = getData("test/y")
test_classes.extend([1 for i in range(len(test_y))])

# read the inverted Y examples and add their class (-1) in the class list
test_y_inv = getData("test/y_inv")
test_classes.extend([-1 for i in range(len(test_y_inv))])

# Concatenate the test_y with test_y_inv examples
test_data = np.concatenate((test_y, test_y_inv))

reading file test/y/y2.csv
reading file test/y/y3.csv
reading file test/y/y1.csv
reading file test/y_inv/y_inv1.csv
reading file test/y_inv/y_inv2.csv
reading file test/y_inv/y_inv3.csv


In [12]:
# The test data
test_data

array([[-1,  1, -1,  1, -1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1,
        -1,  1, -1, -1, -1, -1,  1, -1,  1],
       [ 1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1,
        -1,  1, -1, -1,  1, -1,  1, -1,  1],
       [ 1, -1, -1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1,
        -1,  1, -1, -1,  1,  1,  1, -1, -1],
       [ 1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1, -1,  1,
        -1, -1, -1,  1,  1,  1, -1, -1,  1],
       [ 1,  1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1,  1, -1, -1, -1,
        -1,  1, -1,  1, -1,  1, -1,  1, -1],
       [-1, -1,  1,  1,  1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1,
         1, -1,  1,  1, -1,  1, -1,  1, -1]])

In [13]:
# and its classes
test_classes

[1, 1, 1, -1, -1, -1]

Uso do Perceptron para classificar exemplos de teste:

In [14]:
predicted_test_classes = perceptron.predict(test_data)

Cálculo da Acurácia para exemplos de teste

In [15]:
test_acc = get_accuracy(predicted_test_classes, test_classes)
print("Acurácia para exemplos de teste: %.2f%%" % (test_acc*100))

Acurácia para exemplos de teste: 100.00%


Foram introduzidos ruídos em algumas imagens de teste para tornar a tarefa de classificá-las mais difícil. Para tal, foram acrescentados +1 fora da região que define o símbolo Y ou Y invertido.

Repare que nesse caso, mesmo com os ruídos, obtemos uma acurácia de 100%. 

Entretanto, treinando novamente o Perceptron, podemos chegar a casos em que nem todas as imagens de teste são classificadas corretamente. Isso ilustra o fato de que, ao treinar o Perceptron, encontra-se **um dos possíveis hiperplanos** separadores das duas classes. Mas até o momento, não impomos nenhuma característica a esse plano. Por exemplo, no algoritmo de aprendizado de máquina SVM, escolheríamos o hiperplano que fique mais ao meio da área de separação entre as duas classes.

No caso do nosso Perceptron, durante o treinamento, encontra-se um hiperplano separador mas não é garantido que esse seja o melhor dos possíveis planos separadores.

Abaixo, vamos treinar novamente um perceptron e utilisá-lo para classificar os mesmos exemplos de teste.

In [18]:
perceptron = Perceptron(5*5)
perceptron.train(train_data, train_classes, 0.5)

predicted_test_classes = perceptron.predict(test_data)
test_acc = get_accuracy(predicted_test_classes, test_classes)
print("Acurácia para exemplos de teste: %.2f%%" % (test_acc*100))

Início do Ciclo 1
	A imagem 0 (classe 1) não  foi classificada corretamente!  Ajustando pesos!
Início do Ciclo 2
Acurácia para exemplos de teste: 83.33%


Vamos analisar qual imagem não foi classificada corretamente:

In [19]:
print("Classes Reais: \t\t", end="")
print(test_classes)

print("Classes Preditas: \t", end="")
print(predicted_test_classes)

different_images = [i for i in range(len(test_classes)) if test_classes[i] != predicted_test_classes[i]]
print("Imagens não classificadas corretamente: ", different_images)

Classes Reais: 		[1, 1, 1, -1, -1, -1]
Classes Preditas: 	[1, 1, -1, -1, -1, -1]
Imagens não classificadas corretamente:  [2]


In [20]:
print("Imagens não classificadas corretamente:")
for image in different_images:
    for i in range(5):
        for j in range(5):
            print("%2d" % test_data[2][i*5 + j], end=" ")
        print()
    print("--------------")

Imagens não classificadas corretamente:
 1 -1 -1 -1  1 
 1 -1 -1 -1  1 
-1  1  1  1 -1 
-1 -1  1 -1 -1 
 1  1  1 -1 -1 
--------------


Repare que a imagem não classificada coretamente possui metade de uma linha de 1 na parte inferior como ruído adicionado à letra Y, formando uma espécie de _Y. Esse ruído foi adicionado porque todos os exemplos estavam sendo classificados corretamente após diferentes treinamentos. Ele é um exemplo difícil de se classificar e é interessante notar que, mesmo assim, após determinados treinamentos, como o primeiro apresentado acima, um único perceptron foi capaz de classificá-lo corretamente.